ML Model to predict winner in a League of Legends match based on picked champions

In [ ]:
# We use the cassiopeia library to make calls to the Riot API
import cassiopeia as cass
from cassiopeia.core import Summoner, MatchHistory, Match
from cassiopeia import Queue, Patch
cass.set_riot_api_key("") # Riot API key: https://developer.riotgames.com/

In [ ]:
# Collect summoner IDs of all players in and above master league
challenger_league = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives,region='NA')
gm_league = cass.get_grandmaster_league(queue=cass.Queue.ranked_solo_fives,region='NA')
master_league = cass.get_master_league(queue=cass.Queue.ranked_solo_fives,region='NA')

In [ ]:
# Set starting point for collecting matches
patch = Patch.from_str("11.22", region='NA')

In [ ]:
all_players = set()
all_games = set()

In [ ]:
# Collect PUUIDs from summoner IDs
for entry in challenger_league:
  all_players.add(entry.summoner.puuid)
for entry in gm_league:
  all_players.add(entry.summoner.puuid)
for entry in master_league:
  all_players.add(entry.summoner.puuid)

In [ ]:
# Collect all games played by players in master, grandmaster and challenger league
for player in all_players:
  for game in cass.get_match_history(region='NA',puuid = player, continent='AMERICAS', begin_time=patch.start,queue=Queue.ranked_solo_fives):
    all_games.add(game.id)

In [ ]:
matches = []
results = []

In [ ]:
# Collect champions and results from each game, players 1-5 are blue team, 6-10 red
for match in all_games:
    match = cass.Match(id = match, continent='AMERICAS', region = 'NA')
    if match.queue == Queue.ranked_solo_fives:
        champions = []
        for player in match.participants[:10]:
            champions.append(player.champion.id)
        matches.append(champions)
        if match.participants[0].stats.win == True:
            results.append(1)
        else:
            results.append(0)
    else:
        print(match.queue)

    

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
onehotblue = []
onehotred = []

In [ ]:
# Turn champions of each team into a one hot encoded list, champion IDs range between 1 and 887
onehotblue = []
onehotred = []
for comp in matches:
    a = np.zeros(shape=(887,))
    b = np.zeros(shape=(887,))
    for hero in comp[:5]:
        a[hero-1] = 1
    for hero in comp[5:]:
        b[hero-1] = 1
    onehotblue.append(a)
    onehotred.append(b)

In [ ]:
# Prepare data
y_train = tf.convert_to_tensor(results[:2500])
blue_train = tf.convert_to_tensor(onehotblue[:2500])
red_train = tf.convert_to_tensor(onehotred[:2500])
y_train = tf.convert_to_tensor(results[:2500])
blue_test = tf.convert_to_tensor(onehotblue[2500:])
red_test = tf.convert_to_tensor(onehotred[2500:])
y_test = tf.convert_to_tensor(results[2500:])

In [ ]:
# Build model, first evaluates each team seperately, then combined
inputsblue = keras.Input(shape=(887,))
blue = layers.Dense(64, activation="relu")(inputsblue)
blue = layers.Dropout(0.25)(blue)
blue = layers.Dense(64, activation="relu")(blue)
outputsblue = layers.Dense(64, activation="relu")(blue)
modelblue = keras.Model(inputs=inputsblue, outputs=outputsblue, name="blue_team")
inputsred = keras.Input(shape=(887,))
red = layers.Dense(64, activation="relu")(inputsred)
red = layers.Dropout(0.25)(red)
red = layers.Dense(64, activation="relu")(red)
outputsred = layers.Dense(64, activation="relu")(red)
modelred = keras.Model(inputs=inputsred, outputs=outputsred, name="red_team")
combined = layers.Concatenate(name="concatenate_layer")([modelred.output, modelblue.output])
combined = layers.Dropout(0.75)(combined)
combined = layers.Dense(64, activation="relu")(combined)
combined = layers.Dropout(0.75)(combined)
combined = layers.Dense(64, activation="relu")(combined)
outsputscombined = layers.Dense(1, activation="sigmoid")(combined)
model = tf.keras.Model(inputs=[modelblue.input, modelred.input],
                         outputs=outsputscombined,
                         name="full_model")
model.summary()

In [ ]:
keras.utils.plot_model(model, "league_model.png", show_shapes=True)

In [ ]:
model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model.fit(x=[blue_train,red_train], y=y_train,steps_per_epoch=len(blue_train) ,validation_data=([blue_test,red_test],y_test),epochs=30,validation_steps=len(blue_test))

In [ ]:
model.summary()

In [ ]:
preds = model.predict([blue_test,red_test])

In [ ]:
# Evaluate how far off each prediction is from label
y = y_test.numpy()
preds = np.squeeze(preds)
print(abs(preds-y).mean())